In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf

In [2]:
import json
import math
import itertools
import random
import sys


In [3]:
conf = SparkConf()
# conf.set("spark.driver.memory", "4g")
# conf.set("spark.executor.memory", "4g")
conf.setMaster('local[8]')
conf.setAppName('Assignment_2')
sc = SparkContext.getOrCreate(conf)

In [4]:
import re
import string

In [ ]:
def create_word_count_dict(word_list):
    count_dict = {}
    for word in word_list:
        if word not in count_dict:
            count_dict[word] = 0
        count_dict[word] += 1
    return count_dict


In [ ]:
def get_tf_scores(word_dict):
    max_freq = max(word_dict.values())
    tf_scores = {}
    for word in word_dict:
        tf_scores[word] = word_dict[word]/max_freq
    return tf_scores


In [ ]:
reviews_json = sc.textFile('asnlib/publicdata/train_review.json').map(json.loads)

In [ ]:
reviews_json.first()

In [ ]:
users_token_dict = reviews_json.map(lambda x: x.get('user_id')).distinct().sortBy(lambda x: x).zipWithIndex().collectAsMap()

In [ ]:
business_token_dict = reviews_json.map(lambda x: x.get('business_id')).distinct().sortBy(lambda x: x).zipWithIndex().collectAsMap()

In [ ]:
users_token_dict

In [ ]:
stop_words = set(word.strip() for word in open("asnlib/publicdata/stopwords"))

In [ ]:
business_word_pairs_rdd = reviews_json \
        .map(lambda x: (x.get('business_id'), x.get('text'))) \
        .flatMap(lambda x: [(x[0], word) for word in re.split(r'[{}]'.format(string.whitespace + string.punctuation), x[1].lower())]) \
        .mapValues(lambda x: x.strip(r'[{}]'.format(string.punctuation + string.digits))) \
        .filter(lambda x: x[1] and x[1] != '' and x[1] not in string.ascii_lowercase and x[1] not in string.digits and x[1] not in stop_words)

In [ ]:
business_word_pairs_rdd.first()

In [ ]:
words_token_dict = business_words.map(lambda x: x[1]).distinct().sortBy(lambda x: x).zipWithIndex().collectAsMap()

In [ ]:
words_token_dict

In [ ]:
business_words_tokenized_pairs = business_words.map(lambda x: (business_token_dict[x[0]], words_token_dict[x[1]]))

In [ ]:
business_review_words = business_words_tokenized_pairs.groupByKey().map(lambda x: (x[0], list(x[1])))

In [ ]:
business_review_word_counts = business_review_words.map(lambda x: (x[0], create_word_count_dict(x[1])))

In [ ]:
business_tf_scores = business_review_word_counts.map(lambda x: (x[0], get_tf_scores(x[1])))

In [ ]:
business_tf_scores.first()

In [ ]:
word_business_occurences = business_words_tokenized_pairs.map(lambda x: (x[1], x[0])).groupByKey().map(lambda x: (x[0], list(set(x[1]))))

In [ ]:
word_business.first()

In [ ]:
word_business_dict = word_business_occurences.collectAsMap()

In [ ]:
business_token_dict['bZMcorDrciRbjdjRyANcjA']

In [ ]:
business_word_idf = word_business_occurences.map(lambda x: (x[0], math.log(len(business_token_dict)/len(x[1]), 2)))

In [ ]:
business_word_idf.take(2)

In [ ]:
business_word_idf_dict = business_word_idf.collectAsMap()

In [ ]:
business_tf_idf = business_tf_scores.map(lambda x: (x[0], [(word, x[1][word]*business_word_idf_dict[word]) for word in x[1]]))

In [ ]:
business_tf_idf.take(1)

In [ ]:
business_tf_idf_dict = business_tf_idf.collectAsMap()

In [ ]:
busine

In [ ]:
sorted(business_tf_idf_dict[4435], key=lambda x: -x[1])

In [ ]:
words_token_dict['financial']

In [ ]:
business_tokenized_profile = business_tf_idf.mapValues(lambda x: [word_score_pair[0] for word_score_pair in sorted(x, key=lambda record: -record[1])[:200]])

In [ ]:
business_tokenized_profile_dict = business_tokenized_profile.collectAsMap()

In [ ]:
user_profile = reviews_json \
        .map(lambda x: (users_token_dict[x.get('user_id')], business_token_dict[x.get('business_id')])) \
        .distinct() \
        .flatMapValues(lambda x: business_tokenized_profile_dict[x]) \
        .groupByKey().mapValues(lambda x: list(set(x)))

In [ ]:
user_profile.first()

In [ ]:
user_tokenized_profile_dict = user_profile.collectAsMap()

In [ ]:
def write_model(model):
    with open('model.txt','w') as file:
        for line in model:
            file.write(json.dumps(line))
    file.close()

In [ ]:
def add_to_model(key_type, model_params):
    additional_model_params = [
            {
            'key_type': key_type,
            'id': id_token,
            'token': model_params[id_token]
        } for id_token in model_params
    ]
    return additional_model_params

In [ ]:
model = []

In [ ]:
model.extend(add_to_model('user_token', users_token_dict))

In [ ]:
model.extend(add_to_model('business_token', business_token_dict))

In [ ]:
model.extend(add_to_model('word_token', words_token_dict))

In [ ]:
model.extend(add_to_model('business_profile', business_tokenized_profile_dict))

In [ ]:
model.extend(add_to_model('user_profile', user_tokenized_profile_dict))

In [ ]:
len(model) == len(users_token_dict) + len(business_token_dict) + len(words_token_dict) + len(business_tokenized_profile_dict) + len(user_tokenized_profile_dict)

In [ ]:
write_model(model)

## Getting profiles from scratch

In [ ]:
user_reviews = reviews_json \
        .map(lambda x: (users_token_dict[x.get('user_id')], x.get('text'))) \
        .flatMap(lambda x: [(x[0], word) for word in re.split(escape_chars, x[1])]) \
        .map(lambda x: (x[0], x[1].lower().strip(string.punctuation)))  \
        .filter(lambda x: x[1] and x[1] != '' and x[1] not in string.ascii_lowercase and x[1] not in string.digits and x[1] not in stop_words) \
        .groupByKey().map(lambda x: (x[0], list(x[1]))) \

In [ ]:
user_review_word_counts = user_reviews.map(lambda x: (x[0], create_word_count_dict(x[1])))

In [ ]:
user_tf_scores = user_review_word_counts.map(lambda x: (x[0], get_tf_scores(x[1])))

In [ ]:
user_tf_scores.first()

# Predict

In [5]:
system_model_params = sc.textFile('model.json')

In [6]:
system_model_params = system_model_params.map(json.loads)

In [7]:
system_model_params.first()

{'key_type': 'user_token', 'id': '---1lKK3aKOuomHnwAkAow', 'token': 0}

In [8]:
user_tokens = system_model_params.filter(lambda x: x['key_type'] == 'user_token').map(lambda x: (x['id'], x['token']))
business_tokens = system_model_params.filter(lambda x: x['key_type'] == 'business_token').map(lambda x: (x['id'], x['token']))
business_profile = system_model_params.filter(lambda x: x['key_type'] == 'business_profile').map(lambda x: (x['id'], x['token']))
user_profile = system_model_params.filter(lambda x: x['key_type'] == 'user_profile').map(lambda x: (x['id'], x['token']))


In [9]:
user_tokens_dict = user_tokens.collectAsMap()
business_tokens_dict = business_tokens.collectAsMap()
business_profile_dict = business_profile.collectAsMap()
user_profile_dict = user_profile.collectAsMap()


In [10]:
inverse_business_token_dict = {token: oid for oid, token in business_tokens_dict.items()}
inverse_user_token_dict = {token: oid for oid, token in user_tokens_dict.items()}




In [11]:
test_pairs = sc.textFile('asnlib/publicdata/test_review.json').map(json.loads).map(lambda x: (user_tokens_dict.get(x['user_id'], None), business_tokens_dict.get(x['business_id'], None))).filter(lambda x: x[0] is not None and x[1] is not None).distinct()

In [12]:
test_pairs.first()

(21326, 5428)

In [13]:
def get_cos_sim(user_profile, business_profile):
    if user_profile and business_profile:
        set_user_profile = set(user_profile)
        set_business_profile = set(business_profile)
        return len(set_user_profile.intersection(set_business_profile))/ math.sqrt(len(set_user_profile)*len(set_business_profile))
    else:
        return 0

In [15]:
sim_sets = test_pairs.map(lambda x: ((x[0], x[1]), get_cos_sim(user_profile_dict.get(x[0], None), business_profile_dict.get(x[1], None))))

In [18]:
sim_sets_filtered = sim_sets.filter(lambda x: x[1]> 0.01)

In [19]:
sim_sets.count()

58473

In [14]:
test_pairs.count()

58473

In [20]:
sim_sets_filtered.count()

58438

In [21]:
if None:
    print(1)
else:
    print(2)

2


In [22]:
sim_sets_filtered.take(100)

[((21398, 1846), 0.08246893781033394),
 ((21990, 9964), 0.14827988690081373),
 ((23002, 7254), 0.06937978040731209),
 ((12023, 6725), 0.23539293971054817),
 ((19162, 6376), 0.1496648158458856),
 ((5986, 4074), 0.13907503268608712),
 ((441, 6991), 0.10577499661388097),
 ((26119, 8827), 0.08024287938728723),
 ((22307, 4133), 0.09016142095519147),
 ((18879, 6961), 0.17185276463055116),
 ((14945, 859), 0.14511181735933543),
 ((25423, 6285), 0.08540515179877906),
 ((6494, 3688), 0.2139912463656557),
 ((10317, 5891), 0.09795053547775788),
 ((1997, 6961), 0.1621277219666288),
 ((985, 9855), 0.11845634833373356),
 ((24880, 7112), 0.08510291089601094),
 ((21087, 4071), 0.11888402575869761),
 ((17671, 3401), 0.22342120624110437),
 ((23174, 9276), 0.1648148801644381),
 ((6010, 6136), 0.1918851827829536),
 ((8031, 3663), 0.4186703212093317),
 ((25786, 4250), 0.11244156828420224),
 ((18737, 2699), 0.09290771864676266),
 ((11176, 4996), 0.14741730291501798),
 ((13476, 6182), 0.1337688834049462),
 ((